In [19]:
import os
import yaml
from pymongo import MongoClient
from dotenv import load_dotenv
import pandas as pd

In [20]:
def flatten(l):
  return [item for sublist in l for item in sublist]

In [21]:
load_dotenv()
# This gets all env variables starting with RIM_
cf = {}
for e in os.environ:
  if e.startswith('RIM_'):
    e_name = e.replace('RIM_', '').lower()
    cf[e_name] = os.getenv(e)
# This tries to convert port numbers to int
cf = {k: (int(v) if v.isnumeric() else v) for k, v in cf.items()}

In [22]:
mc = MongoClient(
  host=cf['mongo_host'],
  port=cf['mongo_port'],
  username=cf['mongo_user'],
  password=cf['mongo_pass'],
)
mc._timeout = 1

In [23]:
with open('../data/item-categories.yaml', 'r') as f:
  categories = flatten(list(yaml.safe_load_all(f)))
categories

[{'_id': 'foodstuff',
  'desc': 'All you can eat or drink',
  'subcats': ['greens', 'meat', 'meals', 'beverage', 'liquors']},
 {'_id': 'greens', 'desc': 'Grows in a garden'},
 {'_id': 'meat', 'desc': 'Says Moooo'},
 {'_id': 'meals', 'desc': 'Is full of ingredients'},
 {'_id': 'beverage', 'desc': 'Is liquid'},
 {'_id': 'liquors', 'desc': 'Makes you dizzy'},
 {'_id': 'weapons',
  'desc': 'Made for killing',
  'subcats': ['melee', 'ranged', 'exotic']},
 {'_id': 'melee',
  'desc': "To look into someone's eyes",
  'subcats': ['pointed', 'blunt', 'edged']},
 {'_id': 'ranged', 'desc': 'For the introverts'},
 {'_id': 'exotic', 'desc': 'Is it a frying pan?'},
 {'_id': 'pointed', 'desc': 'Spears'},
 {'_id': 'blunt', 'desc': 'Crushes limbs'},
 {'_id': 'edged', 'desc': 'Sharp as razor'},
 {'_id': 'money',
  'desc': 'The medium of exchange',
  'subcats': ['coins', 'banknotes', 'credit', 'valuables']},
 {'_id': 'coins', 'desc': 'Usually round'},
 {'_id': 'banknotes', 'desc': 'Paper shreds'},
 {'_id'

In [24]:
df = pd.DataFrame(categories)
parents = []
for _, row in df.iterrows():
  if type(row['subcats']) == list:
    for cat in row['subcats']:
      parents.append({
        '_id': cat,
        'parent': row['_id']
      })
parents = pd.DataFrame(parents)
categories = pd.concat([df.set_index('_id'), parents.set_index('_id')], axis=1, join='outer').reset_index()
categories

,_id,desc,subcats,parent
0,foodstuff,All you can eat or drink,"[greens, meat, meals, beverage, liquors]",NaN
1,greens,Grows in a garden,NaN,foodstuff
2,meat,Says Moooo,NaN,foodstuff
3,meals,Is full of ingredients,NaN,foodstuff
4,beverage,Is liquid,NaN,foodstuff
5,liquors,Makes you dizzy,NaN,foodstuff
6,weapons,Made for killing,"[melee, ranged, exotic]",NaN
7,melee,To look into someone's eyes,"[pointed, blunt, edged]",weapons
8,ranged,For the introverts,NaN,weapons
9,exotic,Is it a frying pan?,NaN,weapons


In [25]:
mc['models'].drop_collection('items_categories')
mc['models']['items_categories'].insert_many(categories.to_dict(orient='records'))

InsertManyResult(['foodstuff', 'greens', 'meat', 'meals', 'beverage', 'liquors', 'weapons', 'melee', 'ranged', 'exotic', 'pointed', 'blunt', 'edged', 'money', 'coins', 'banknotes', 'credit', 'valuables', 'wearables', 'clothing', 'armor', 'accessories', 'hazmat', 'pants', 'jackets', 'shirts', 'hats', 'armor vests', 'armor helmets', 'armor plates', 'armor suit', 'armor exo'], acknowledged=True)

In [26]:
with open('../data/items.yaml', 'r') as f:
  items = flatten(list(yaml.safe_load_all(f)))
items

[{'_id': 'Cucumber',
  'cat': 'greens',
  'desc': 'Makes cats scared',
  'opts': ['weight', 'quality', 'shape']},
 {'_id': 'Tomato',
  'cat': 'greens',
  'desc': 'Is not green',
  'opts': ['weight', 'quality', 'color']},
 {'_id': 'Beef',
  'cat': 'meat',
  'desc': 'Once it said Mooo last time',
  'opts': ['weight', 'quality', 'part']}]

In [27]:
items_sorted = {}
for v in items:
  vc = v.copy()
  del vc['cat']
  try:
    items_sorted[v['cat']].append(vc)
  except:
    items_sorted[v['cat']] = [vc]
items_sorted

{'greens': [{'_id': 'Cucumber',
   'desc': 'Makes cats scared',
   'opts': ['weight', 'quality', 'shape']},
  {'_id': 'Tomato',
   'desc': 'Is not green',
   'opts': ['weight', 'quality', 'color']}],
 'meat': [{'_id': 'Beef',
   'desc': 'Once it said Mooo last time',
   'opts': ['weight', 'quality', 'part']}]}

In [28]:
for k, v in items_sorted.items():
  mc['items'].drop_collection(k)
  mc['items'][k].insert_many(v)